# Downloading and coverting ERA5 files

Information on the fifth generation ECMWF atmospheric reanalysis of the global climate ERA5 is found at:
https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview

# 1) Requesting ERA5 data

a) Go to the ERA5 hourly data on single levels from 1979 to present (https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=form)

b) In product type select Reanalysis

c) In Variable select Popular -> 10m u-component of wind, 10m v-component of wind, and Mean sea level pressure.

d) In this example we will prepare the data for Hurricane Arthur, from 06/27/2014 to 07/09/2014. Therefore, we will have to download 2 ERA5 files, one from 06/27/2014 to 06/30/2014 and another from 07/01/2014 to 07/09/2014. 

First, we will download the data from 06/27/2014 to 06/30/2014. To do that we just need to select the Year, Month and Day accordingly. We will work with inputs every 6 hours, so select the Time: 00:00, 06:00, 12:00, and 18:00.

e) In the Geographical area tab select “Whole available region”

f) Select the GRIB format in the Format tab.

g) Click on “Show API request” and save the code.


# 2) Downloading ERA5 data

Importing the necessary lybraries:


cfgrib is installed as:
conda install -c conda-forge cfgrib
xarray is installed as:
conda install -c conda-forge xarray dask netCDF4 bottleneck

In [1]:
import sys;sys.path.append('../adcirc_swan')
import arctic as arc;
import sys,getopt
import pandas as pd
import numpy as np
import pathlib as pl
import numpy.core.multiarray 
import netCDF4 as nc4
import xarray as xr
import scipy.interpolate
import cdsapi
import datetime
from pyproj import Proj, transform
import matplotlib.pyplot as plt;
c = cdsapi.Client()
from PIL import Image
import matplotlib as mpl;#import geopandas as gpd

Paste the API request and start downloading:

In [2]:
days = np.arange(0,31).astype(str).tolist()
#years = np.arange(1994,2022).astype(str).tolist()
#months= np.arange(1,12).astype(str).tolist()

In [3]:
time = ['00:00', '02:00', '04:00','06:00', '08:00', '10:00',
        '12:00', '14:00', '16:00','18:00', '20:00', '22:00',]
        #'12:00', '13:00', '14:00','15:00', '16:00', '17:00',
        #'18:00', '19:00', '20:00','21:00', '22:00', '23:00',]
        #
c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format':'netcdf',
        #'area':[90,-180,30,180],
        'variable': [
            '10m_u_component_of_wind',
            '10m_v_component_of_wind',
            'mean_sea_level_pressure',
            'Sea_Ice_Concentration',
        ],
        'year': '2019',
        'month': ['10','11','12'],
        'day': days,
        'time': time,
    },'storm2019.nc')

2023-03-02 15:10:50,796 INFO Welcome to the CDS
2023-03-02 15:10:50,797 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-03-02 15:10:51,002 INFO Request is queued
2023-03-02 15:10:52,143 INFO Request is running
2023-03-02 15:23:12,976 INFO Request is completed
2023-03-02 15:23:12,977 INFO Downloading https://download-0016.copernicus-climate.eu/cache-compute-0016/cache/data0/adaptor.mars.internal-1677788011.3052394-2133-4-a76316af-1df6-4ae3-9a75-fbefee76690d.nc to storm2019.nc (8.4G)
2023-03-02 15:52:51,332 INFO Download rate 4.8M/s                                                                                                               


Result(content_length=8970408508,content_type=application/x-netcdf,location=https://download-0016.copernicus-climate.eu/cache-compute-0016/cache/data0/adaptor.mars.internal-1677788011.3052394-2133-4-a76316af-1df6-4ae3-9a75-fbefee76690d.nc)

# 3) Converting the .grib file into .22 (NWS=6)

Set the path to where the .grib data was saved:

Open the .grib file:

In [5]:
path = pl.Path('/Users/tmiesse/work/FHRL/arctic/model/ice_paper/storm2019/sea_ice/era5')
file = 'storm2019.nc'
grib = xr.open_dataset(path / file)#, engine='cfgrib')
#file = 'wnd10m.gdas.201911.grib2'
#grib2= xr.open_dataset(path / file, engine='cfgrib')
#lats2 = np.arange(90,-90.205,-0.205)
#lons2 = np.arange(0,360.205,0.205)
#grib2 = grib2.interp(latitude=lat,longitude=lon,method='nearest')#.data_vars['prmsl']

Extract the variables of interest:

In [6]:
# bounding box of domain
lat1,lat2 = 30,90
lon1,lon2 = -220, -84

In [14]:
x,y = grib['xgrid'],grib['ygrid']
xgrid,ygrid = np.meshgrid(x,y)

In [16]:
inProj = Proj(init='epsg:3411')
outProj = Proj(init='epsg:4326')
x2,y2 = transform(inProj,outProj,xgrid,ygrid)

/Users/tmiesse/miniconda3/envs/general/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

/Users/tmiesse/miniconda3/envs/general/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

/var/folders/6c/dqn6lv8s37986zd3jwzl322w0000gn/T/ipykernel_82370/734846631.py:3: DeprecationWarning:

This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1



In [7]:
lat = grib.coords['latitude'][:].values
lon= grib.coords['longitude'][:].values
time = grib.coords['time'].values[:]
idx = np.where((lon1+360 <= lon)&(lon<=lon2+360))[0]
idy = np.where((lat1 <= lat)&(lat<=lat2))[0]


In [8]:
ts = [(t - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')  for t in time]
ts2 = [datetime.datetime.utcfromtimestamp(t) for t in ts]

/var/folders/6c/dqn6lv8s37986zd3jwzl322w0000gn/T/ipykernel_8159/880946532.py:1: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future



In [9]:
ts2[-1]

datetime.datetime(2019, 12, 30, 22, 0)

In [12]:
#ts = [(t - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')  for t in time]
#ts2 = [datetime.datetime.utcfromtimestamp(t) for t in ts]
#ts2 = time
dx = 0.25#np.abs(lon[0]-lon[1])# 0.205 is cfsv2 wind increment
dy = 0.25#np.abs(lat[0]-lat[1])#
ilon = len(idx)
ilat = len(idy)
start = ts2[0]
end = ts2[-1]
swlat = lat1
swlon = lon1#+360
header = 'Oceanweather WIN/PRE Format                            {:12s}   {:12s}\n'.format(str(start.strftime('%Y%m%d%H')),str(end.strftime('%Y%m%d%H')))#     {}'
dt = ts2
file = 'fort.225'

arc.force2owi(file,dx,dy,dt,ilon,ilat,start,end,swlat,swlon,header,grib,lat1,lat2,lon1,lon2,\
          uwind='u10',vwind='v10',psl='msl',forcing='era5')

amount of data 131345 data is 131345
generated owi formatted forcings


In [2]:
path = pl.Path('/Users/tmiesse/work/FHRL/arctic/ice_data/satelite/sentinel-2')
files = list(path.glob('*'))
lat1,lat2 = 30,90
lon1,lon2 = -250, -85
start = pd.to_datetime(files[0].name.split('_')[4])
end = pd.to_datetime(files[-1].name.split('_')[4])
header = 'Oceanweather WIN/PRE Format                            {:12s}   {:12s}\n'.format(str(start.strftime('%Y%m%d%H')),str(end.strftime('%Y%m%d%H')))#     {}'

inProj = Proj(init='epsg:3411')
outProj = Proj(init='epsg:4326')


/Users/tmiesse/miniconda3/envs/general/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

/Users/tmiesse/miniconda3/envs/general/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



In [3]:
arc.sat_seaice2owi(header,files[:1],inProj,outProj,lat1,lat2,lon1,lon2,sea_ice='cdr_seaice_conc')

amount of data 5020990464 data is 9
generated owi formatted forcings


'20190104'